This notebook uses compiled version of the computeVPB function. To compile it run the command

    python setup.py build_ext --inplace

from the `python/` directory

See https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html for some basic tutorial

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import ripser
import persim


In [2]:
import TDAvec
from TDAvec import DiagToPD, computeVPB, computePL

In [3]:
X = np.loadtxt("../R/unitCircle.csv", skiprows=1, delimiter=",")
D = ripser.ripser(X, thresh=2)["dgms"]
D[0][-1, 1] = 2


## computeVPV

In [8]:
PD = DiagToPD(D)
ySeqH0 = np.quantile(PD[0][1], np.arange(0, 1.1, 0.2))
xSeqH1 = np.quantile(PD[1][0], np.arange(0, 1.1, 0.2))
ySeqH1 = np.quantile(PD[1][1], np.arange(0, 1.1, 0.2))

vpb0 = computeVPB(PD, homDim=0, xSeq=[], ySeq=ySeqH0)
vpb1 = computeVPB(PD, homDim = 1, xSeq=xSeqH1, ySeq=ySeqH1)

In [9]:
np.allclose(vpb0, [0.809001, 3.644815, 5.486612, 6.382816, 1.013953])

True

In [27]:
vpb_ = np.transpose(vpb1).reshape( (25,))
vpbR = [ 0.0000000000, 0.0004995598, 0.0090337733, 0.0305098818, 0.0057973687, 0.0095194496, 0.0261053157, 0.0203673036, 0.0042870130,
        0.0107446149, 0.0425768331, 0.1053468321, 0.1458807241, 0.0000000000, 0.0000000000, 0.0328035481, 0.0276785592, 0.1089088594,
        0.1318822254, 0.0168304895, 0.2939394561, 0.3162986126, 0.3344510989, 0.3567486932, 0.3636226941]
max(abs(vpb_ - vpbR))
np.allclose(vpb_,vpbR, atol = 1e-7)

True

In [31]:
scaleSeq = np.linspace(0, 2, 11)
print(np.allclose( computePL(D, 0, scaleSeq), [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 0.8, 0.6, 0.4, 0.2, 0.0]))
print(np.allclose(computePL(D, 1, scaleSeq), [0.000000000, 0.014217630, 0.000931983, 0.191114536,
                                         0.391114536, 0.269212150, 0.069212150,
                                           0.000000000, 0.000000000, 0.000000000, 0.000000000]))

True
True
